In [2]:
import os
from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("Doogie.2ndKey")
os.environ['OPENAI_API_KEY'] = api_key

In [3]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import PromptTemplate
from langchain_core.tracers import ConsoleCallbackHandler
from langchain_community.chat_message_histories import ChatMessageHistory

In [5]:
from langchain_openai import ChatOpenAI

myllm = ChatOpenAI(
    temperature=0.1,
    max_tokens=2048,
    model='gpt-3.5-turbo',
)

question = "세종대왕이 누구인지 설명해줘"

result = myllm.invoke( question )
print(result.content)
#print(result)

세종대왕은 조선시대 4대 왕 중 한 명으로, 조선시대의 제4대 군주로 알려져 있습니다. 그의 본명은 이도이며, 세종대왕은 1418년에서 1450년 사이에 통치했습니다. 세종대왕은 조선시대의 중요한 업적을 남겼는데, 특히 한글을 창제하고 조선의 문화와 교육을 발전시킨 것으로 유명합니다. 또한 세종대왕은 과학기술 발전과 국방력 강화에도 많은 노력을 기울였습니다. 그의 통치는 조선시대의 전성기를 이루는 데 큰 역할을 했습니다.


In [6]:
from langchain_core.prompts import PromptTemplate

mytemplate = "{who}가 누구인지 설명해 줘."

myprompt = PromptTemplate(
    template=mytemplate, input_variables={'who'}
)

print(myprompt)
print(myprompt.format(who="오바마"))

input_variables=['who'] input_types={} partial_variables={} template='{who}가 누구인지 설명해 줘.'
오바마가 누구인지 설명해 줘.


In [7]:
result = myllm.invoke( myprompt.format(who="오바마") )
print(result.content)

바락 오바마는 미국의 44대 대통령으로, 2009년부터 2017년까지 두 번의 대통령 임기를 이행했습니다. 그는 미국 역사상 첫 번째 흑인 대통령으로, 미국 역사상 가장 영향력 있는 인물 중 하나로 꼽힙니다. 오바마 대통령은 특히 보헤미안 랩시디를 통해 미국의 건강 보험 개혁, 기후 변화 대책, 동성 결혼 합법화 등 다양한 사회 정책을 추진했습니다. 그의 대통령 시기는 "예스 위 캔"이라는 슬로건으로 유명하며, 많은 사람들에게 영향을 미쳤습니다. 현재 그는 미국의 정치적인 활동을 계속하고 있습니다.


In [8]:
myllm_chain = myprompt | myllm
result = myllm_chain.invoke( {"who": "이순신장군"} )
print(result.content)
result

이순신 장군은 조선시대의 무신이자 민족영웅으로, 16세기 말부터 17세기 초까지 활약한 조선의 장군입니다. 이순신은 일본의 침략을 막기 위해 전략적으로 뛰어난 전투를 펼치며 일본의 대군을 물리쳤고, 특히 함평해전, 명량해전 등의 전투에서 큰 승리를 거두었습니다. 이순신은 뛰어난 전략가로서 뿐만 아니라 뛰어난 해전 전투 능력을 지녔으며, 그의 행적은 한국 역사상 가장 위대한 장군 중 하나로 평가받고 있습니다. 현재까지도 그의 업적은 많은 사람들에게 존경과 찬양을 받고 있습니다.


AIMessage(content='이순신 장군은 조선시대의 무신이자 민족영웅으로, 16세기 말부터 17세기 초까지 활약한 조선의 장군입니다. 이순신은 일본의 침략을 막기 위해 전략적으로 뛰어난 전투를 펼치며 일본의 대군을 물리쳤고, 특히 함평해전, 명량해전 등의 전투에서 큰 승리를 거두었습니다. 이순신은 뛰어난 전략가로서 뿐만 아니라 뛰어난 해전 전투 능력을 지녔으며, 그의 행적은 한국 역사상 가장 위대한 장군 중 하나로 평가받고 있습니다. 현재까지도 그의 업적은 많은 사람들에게 존경과 찬양을 받고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 281, 'prompt_tokens': 28, 'total_tokens': 309, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvgKVzKfchMjh9d0a7JDtMUr3ELic', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3badfd5e-2563-49e3-8aa2-b957716dea44-0', usage_metadata={'input_tokens': 28, 'output_tokens': 281, 'total_tokens': 309, 'input_token_details': {'audio': 0, 'cache_read': 0}

In [9]:
from langchain_core.tracers import ConsoleCallbackHandler

result = myllm_chain.invoke(
    {"who": "강감찬장군"},
    config={"callbacks": [ConsoleCallbackHandler()]},
)
print(result.content)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "who": "강감찬장군"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "who": "강감찬장군"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 강감찬장군가 누구인지 설명해 줘."
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [3.00s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "강감찬(姜邯贊, 925년 ~ 975년)은 고려시대의 무신이자 장군으로, 고려의 제2대 임금인 고종의 시기에 활약했습니다. 고려의 건국 이후 처음으로 외국인 침입을 막아낸 영웅으로, 고려의 국력을 회복하고 국경을 방어하는 데 큰 기여를 했습니다.\n\n강감찬은 소국의 출신으로, 어려서부터 뛰어난 무예 실력을 보였고, 고려의 군사 교육을 받아 뛰어난 장군으로 성장했습니다. 특히, 고려와 후삼국 연합군이 싸우는 고려 후삼국 전쟁에서 큰 승리를 거둬 고려의 국력을 회복시키는 데 큰 역할을 했습니다.\n\n강감찬은 또한 고려의 국방을 강화하고 국경을 방어하기 위해 다양한 방법을 시도했습니다. 그의 뛰어난 전략과 무예 실력으로 많은 전투에서 승리를 거두었고, 그의 행적은 고려시대의 역사에 큰 흔적을 남겼습니다. 현재까지도 그의 이름

In [4]:
from langchain_core.tracers import ConsoleCallbackHandler

result = myllm_chain.invoke(
    {"who": "강감찬장군"},
    config={"callback": ConsoleCallbackHandler() }, 
)
print(result.content)


NameError: name 'myllm_chain' is not defined

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory

mytemplate = """
    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이봇 입니다.
    대화 문맥을 바탕으로 친정한 답변을 진행해라

    Current Conversation:
    {myhistory}

    Human: {myinput}
    AI:
"""
myprompt = PromptTemplate(
    template=mytemplate, input_variables={'myhistory', 'myinput'}
)

myllm = ChatOpenAI( model='gpt-3.5-turbo',)
mychain = myprompt | myllm

mystore = {}
session_id = "myFirstSession"

if session_id not in mystore:
    mystore[session_id] = ChatMessageHistory()
mysessionhistory = mystore[session_id]

mymsghistory = RunnableWithMessageHistory(
    mychain,
    lambda session_id: mysessionhistory,
    input_messages_key="myinput",
    history_messages_key="myhistory"
)

result = mymsghistory.invoke(
    {"myinput": "너는 어디에서 만들었는가?"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print(result.content)

result = mymsghistory.invoke(
    {"myinput": "너의 이름은 무엇인가?"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print(result.content)

In [28]:
result = mymsghistory.invoke(
    {"myinput": "푸른 바다를 주제로 해학과 중의가 넘치는 짧은 시를 만들어줘"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print(result.content)

너무 아름다운 푸른 바다
물결이 춤을 추며 흐른다
바람과 함께 자유롭게 춤을 추는
바다 속의 물고기들의 행복한 노래


In [30]:
result = mymsghistory.invoke(
    {"myinput": "석양을 주제로도 해줘"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print(result.content)

석양의 황금빛이 바다 위로 노래를 부르며 살랑살랑 흔든다. 저기 저 바다를 향해 선배들이 돌아가고 있어. 종소리와 함께 하루가 저물고, 저 멀리 솟아오르는 달이 조용히 반짝이고 있어.


In [39]:
print("mystore:", len(mystore.keys() ) )

mystore: 1


In [ ]:
print("mytemplate:", mytemplate)
print("myprompt:", myprompt)
print("myllm:", myllm)
print("mychain:", mychain)
print("mystore:", mystore)
print("session_id:", session_id)
print("mystore[",session_id,"]=", mystore )
print("mysessionhistory:", mysessionhistory)
